# Get Dataset

In [ ]:
# Get roboflow data for pickleball court keypoints detection training
# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="KEYHERE")
# project = rf.workspace("pickleball-ball-detection").project("pickleball-court-keypoints-syncz")
# version = project.version(6)
# dataset = version.download("coco")

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="KEYHERE")
project = rf.workspace("pickleball-mskux").project("court-detection-bxo2j-npaqa")
version = project.version(1)
dataset = version.download("yolov8")
                

python(68915) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
loading Roboflow workspace...
loading Roboflow project...


# Start Code

In [38]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create Torch Dataset

In [40]:
class keyPointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h, w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        # Adjust keypoints
        kps[::2] *= (224 / w) # Adjust x coordinates
        kps[1::2] *= (224 / h) # Adjust y coordinates

        return img, kps
    

In [41]:
# --- YOLOv8 -> flat adapter for your keyPointsDataset (no changes to your dataset class) ---
from pathlib import Path
import json, cv2

# Point to the YOLOv8 export directories from Roboflow
root = Path("Court-Detection-1")  # adjust if your folder name differs
train_dir = root / "train"
val_dir   = root / ("valid" if (root / "valid").exists() else "val")

rf_train_images = train_dir / "images"
rf_train_labels = train_dir / "labels"
rf_val_images   = val_dir / "images"
rf_val_labels   = val_dir / "labels"

# Output (what your dataset will consume)
out_img_dir   = Path("data_idx/images")
out_img_dir.mkdir(parents=True, exist_ok=True)
out_train_json = Path("data_idx/data_train_flat.json")
out_val_json   = Path("data_idx/data_val_flat.json")

def parse_yolo_kps(img_dir: Path, lbl_dir: Path, out_img_dir: Path, start_idx=0, num_kps=12):
    """
    Reads YOLOv8 keypoints labels and writes:
      - images: data_idx/images/{id}.png
      - items:  [{'id': id, 'kps': [x1,y1,...,xN,yN]}]
    Chooses one annotation per image (the one with most visible kps).
    """
    items = []
    idx = start_idx

    # iterate over label files
    for lbl_path in sorted(lbl_dir.glob("*.txt")):
        stem = lbl_path.stem

        # find matching image (common extensions)
        img_path = None
        for ext in (".jpg", ".jpeg", ".png", ".JPG", ".PNG"):
            p = img_dir / f"{stem}{ext}"
            if p.exists():
                img_path = p
                break
        if img_path is None:
            continue

        with open(lbl_path) as f:
            lines = [ln.strip() for ln in f if ln.strip()]

        if not lines:
            continue

        # pick best line by # visible kps (v>0)
        best = None
        best_vis = -1
        for ln in lines:
            parts = ln.split()
            # cls(1) + bbox(4) + (x,y,v)*num_kps
            if len(parts) < 5 + 3 * num_kps:
                continue
            nums = list(map(float, parts))
            kps = nums[5:]  # x1 y1 v1 x2 y2 v2 ...
            vis = sum(1 for j in range(2, len(kps), 3) if kps[j] > 0)
            if vis > best_vis:
                best_vis = vis
                best = nums

        if best is None:
            continue

        img = cv2.imread(str(img_path))
        if img is None:
            continue
        h, w = img.shape[:2]

        k = best[5:]
        xy = []
        for j in range(0, 3 * num_kps, 3):
            x_abs = k[j] * w
            y_abs = k[j + 1] * h
            xy.extend([float(x_abs), float(y_abs)])

        out_path = out_img_dir / f"{idx}.png"
        cv2.imwrite(str(out_path), img)

        items.append({"id": idx, "kps": xy})
        idx += 1

    return items, idx

# Build flat train/val
train_items, nxt = parse_yolo_kps(rf_train_images, rf_train_labels, out_img_dir, start_idx=0, num_kps=12)
with open(out_train_json, "w") as f:
    json.dump(train_items, f)
print(f"FLAT train: {len(train_items)} items -> {out_train_json}, images in {out_img_dir}")

val_items, _ = parse_yolo_kps(rf_val_images, rf_val_labels, out_img_dir, start_idx=nxt, num_kps=12)
with open(out_val_json, "w") as f:
    json.dump(val_items, f)
print(f"FLAT val:   {len(val_items)} items -> {out_val_json}")


FLAT train: 568 items -> data_idx/data_train_flat.json, images in data_idx/images
FLAT val:   0 items -> data_idx/data_val_flat.json


In [42]:
from pathlib import Path

train_images = Path("data_idx/images")
val_images   = Path("data_idx/images")            # shared pool is fine
train_ann    = Path("data_idx/data_train_flat.json")
val_ann      = Path("data_idx/data_val_flat.json")

In [44]:
# Fallback: ensure val has items; if empty/missing, clone a small subset of train
from pathlib import Path
import json

def _len_json(p):
    try:
        with open(p) as f:
            data = json.load(f)
        return len(data) if isinstance(data, list) else 0
    except Exception:
        return 0

train_count = _len_json(train_ann)
val_count   = _len_json(val_ann)

if val_count == 0:
    # make a small validation set from train (10% or at least 32, capped at train size)
    with open(train_ann) as f:
        train_items = json.load(f)
    n = min(train_count, max(32, int(0.10 * train_count)))
    with open(val_ann, "w") as f:
        json.dump(train_items[:n], f)
    print(f"[fallback] Created val set with {n} items at {val_ann}")
else:
    print(f"val set OK: {val_count} items")


[fallback] Created val set with 56 items at data_idx/data_val_flat.json


In [45]:
# Obtain datasets
train_dataset = keyPointsDataset(str(train_images), str(train_ann))
val_dataset   = keyPointsDataset(str(val_images), str(val_ann))

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=8, shuffle=True)

# Create Model

In [46]:
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 12*2) # Replaces the last layer for 12 keypoints (x, y) pairs

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.


In [47]:
# Move model to device
model = model.to(device)

# Train Model

In [48]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [49]:
epochs = 20
for epoch in range(epochs):
    for i, (imgs, kps) in enumerate(train_loader):
        imgs, kps = imgs.to(device), kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item()}")

Epoch [1/20], Step [1/71], Loss: 15029.46875
Epoch [1/20], Step [11/71], Loss: 16209.1044921875
Epoch [1/20], Step [21/71], Loss: 15634.1923828125
Epoch [1/20], Step [31/71], Loss: 14280.328125
Epoch [1/20], Step [41/71], Loss: 15001.8076171875
Epoch [1/20], Step [51/71], Loss: 14506.8134765625
Epoch [1/20], Step [61/71], Loss: 13201.4990234375
Epoch [1/20], Step [71/71], Loss: 12606.6240234375
Epoch [2/20], Step [1/71], Loss: 13177.25390625
Epoch [2/20], Step [11/71], Loss: 13829.2783203125
Epoch [2/20], Step [21/71], Loss: 11905.25390625
Epoch [2/20], Step [31/71], Loss: 11352.3408203125
Epoch [2/20], Step [41/71], Loss: 11444.14453125
Epoch [2/20], Step [51/71], Loss: 12211.4775390625
Epoch [2/20], Step [61/71], Loss: 11875.9052734375
Epoch [2/20], Step [71/71], Loss: 10319.080078125
Epoch [3/20], Step [1/71], Loss: 11012.3330078125
Epoch [3/20], Step [11/71], Loss: 10109.1435546875
Epoch [3/20], Step [21/71], Loss: 10193.7841796875
Epoch [3/20], Step [31/71], Loss: 9747.9267578125


In [51]:
# Save the model
torch.save(model.state_dict(), "keypoint_model2.pth")